In [1]:
import kidgarden
import numpy as np
import os

num_grids = kidgarden.get_number_grids_from_image("e2.png")
matrix = kidgarden.build_matrix_from_grid_imgs(num_grids)
matrix = np.array(matrix)
print(matrix)

[[9 8 2 4 1 9 6 6 7 7]
 [9 1 3 5 8 2 5 9 3 9]
 [1 2 5 6 9 5 3 6 4 8]
 [3 4 1 4 2 4 9 5 7 2]
 [3 3 6 4 2 1 6 6 3 1]
 [4 4 6 2 2 3 9 5 4 9]
 [9 9 5 3 9 4 8 5 8 5]
 [1 6 7 6 8 9 5 4 1 7]
 [2 7 7 5 4 1 4 3 4 3]
 [9 9 2 6 8 2 6 6 8 7]
 [8 2 9 7 6 6 8 2 8 8]
 [8 9 4 5 1 2 6 5 2 3]
 [2 1 8 4 7 5 7 5 6 8]
 [2 3 4 2 7 1 2 7 3 2]
 [1 2 8 5 9 7 5 1 8 2]
 [2 3 4 7 2 8 9 6 7 5]]


In [2]:
from colorama import Fore, Back, Style

In [3]:
def print_with_color(matrix, rect:tuple):
    in_rect = lambda i,j: i >= rect[0] and i < rect[0] + rect[2] and j >= rect[1] and j < rect[1] + rect[3]
    out_str = ""
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            print_value = " " if matrix[i][j] == 0 else matrix[i][j]
            if in_rect(i, j):
                out_str += f"{Back.RED}{print_value}{Style.RESET_ALL} "
            else:
                out_str += f"{print_value} "
        out_str += "\n"
    
    print(out_str)


In [4]:
print_with_color(matrix=matrix, rect=(2,5,3,2))

9 8 2 4 1 9 6 6 7 7 
9 1 3 5 8 2 5 9 3 9 
1 2 5 6 9 5 3 6 4 8 
3 4 1 4 2 4 9 5 7 2 
3 3 6 4 2 1 6 6 3 1 
4 4 6 2 2 3 9 5 4 9 
9 9 5 3 9 4 8 5 8 5 
1 6 7 6 8 9 5 4 1 7 
2 7 7 5 4 1 4 3 4 3 
9 9 2 6 8 2 6 6 8 7 
8 2 9 7 6 6 8 2 8 8 
8 9 4 5 1 2 6 5 2 3 
2 1 8 4 7 5 7 5 6 8 
2 3 4 2 7 1 2 7 3 2 
1 2 8 5 9 7 5 1 8 2 
2 3 4 7 2 8 9 6 7 5 



In [5]:
def rect_sum(matrix, rect:tuple):
    sub_matrix = matrix[rect[0]:rect[0] + rect[2], rect[1]: rect[1] + rect[3]]
    return np.sum(sub_matrix)

In [16]:
def execute_rect(matrix, rect):
    y, x, h, w = rect
    matrix[y : y + h, x : x + w] = 0

    return matrix

In [6]:
def find_all_matches(matrix):
    rows, cols = len(matrix), len(matrix[0])
    matched_rects = []

    for i in range(rows):
        for j in range(cols):
            height_max = False
            for h in range(1, rows - i):
                for w in range(1, cols - j):
                    if h == 1 and w == 1:
                        continue
                    rect = (i,j,h,w)
                    now_sum = rect_sum(matrix, rect)
                    if now_sum == 10:
                        matched_rects.append(rect)
                    if now_sum >= 10:
                        if w == 1:
                            height_max = True
                        break
                if height_max == True:
                    break
    return matched_rects

In [17]:
def find_all_seq_matches(origin_matrix:np.ndarray, shuffle_index:bool = True):
    r'''
    此方法和上一方法的区别在于，此方法寻找的是在不重叠情况下， 一次可以并行消除的所有区域，这样虽然压缩了可能性，但是能有效剪枝
    '''
    matrix = np.copy(origin_matrix)
    rows, cols = len(matrix), len(matrix[0])
    matched_rects = []

    rows_list = list(range(rows))
    cols_list = list(range(cols))

    if shuffle_index:
        np.random.shuffle(rows_list)
        np.random.shuffle(cols_list)
    
    for i in rows_list:
        for j in cols_list:
            height_max = False
            for h in range(1, rows - i):
                for w in range(1, cols - j):
                    if h == 1 and w == 1:
                        continue
                    rect = (i,j,h,w)
                    now_sum = rect_sum(matrix, rect)
                    if now_sum == 10:
                        matched_rects.append(rect)
                        execute_rect(matrix, rect)
                    if now_sum >= 10:
                        if w == 1:
                            height_max = True
                        break
                if height_max == True:
                    break
    return matched_rects

In [8]:
def calc_score(matrix):
    return np.count_nonzero(matrix == 0)

## 广搜思路：只记录原始matrix, current_matrix，rects，rects执行完毕的分数记录即可

In [9]:
# steps结构 [current_matrix, rects, score]
class Solution:
    def __init__(self, origin_matrix:np.ndarray):
        self.current_matrix = np.copy(origin_matrix)
        self.rects = []
        self.score = 0
        self.max_score = self.current_matrix.size

    def add_step(self, rect):
        self.rects.append(rect)
        y, x, h, w = rect
        self.score += np.count_nonzero(self.current_matrix[y : y + h, x : x + w])
        execute_rect(self.current_matrix, rect=rect)

    def is_win(self):
        return self.score == self.max_score

    def clone(self):
        new_step = Solution(self.current_matrix)
        new_step.rects = [rect for rect in self.rects]
        new_step.score = self.score
        return new_step


In [21]:
from collections import deque
origin_solution = Solution(matrix)
solutions_queue = deque([origin_solution])

best_solution = origin_solution
calc_count = 0

branches = 1
while len(solutions_queue) > 0:
    
    solution = solutions_queue.pop()
    
    next_rects_list = [find_all_seq_matches(solution.current_matrix) for _ in range(branches)]
    for rects in next_rects_list:
        new_solution = solution.clone()
        calc_count += 1
        for rect in rects:
            new_solution.add_step(rect)
        if new_solution.is_win():
            best_solution = new_solution
            break
        if new_solution.score > best_solution.score:
            best_solution = new_solution
            print(f"Calc {calc_count} times, Now Score:{best_solution.score}, QueueSize:{len(solutions_queue)}")
        # print(new_solution.score, end=' ')
        solutions_queue.appendleft(new_solution)
        print(f"QSize:{len(solutions_queue)}")
    
    if best_solution.is_win():
        print("Win Solution Found")
        break

print(f"Best Score: {best_solution.score}")
input("Press to show step")

display_matrix = np.copy(matrix)
for rect in best_solution.rects:
    print_with_color(display_matrix, rect)
    execute_rect(display_matrix, rect)
    input("Press for next")

Calc 1 times, Now Score:82, QueueSize:0
QSize:1
QSize:2
QSize:3
Calc 4 times, Now Score:88, QueueSize:2
QSize:3
Calc 5 times, Now Score:91, QueueSize:3
QSize:4
QSize:5
QSize:5
QSize:6
QSize:7
QSize:7
QSize:8
QSize:9
QSize:9
QSize:10
QSize:11
QSize:11
QSize:12
QSize:13
QSize:13
QSize:14
QSize:15
QSize:15
QSize:16
QSize:17
QSize:17
QSize:18
QSize:19
QSize:19
QSize:20
QSize:21
QSize:21
QSize:22
QSize:23
QSize:23
QSize:24
QSize:25
QSize:25
QSize:26
QSize:27
QSize:27
QSize:28
QSize:29
QSize:29
QSize:30
QSize:31
QSize:31
QSize:32
QSize:33
QSize:33
QSize:34
QSize:35
QSize:35
QSize:36
QSize:37
QSize:37
QSize:38
QSize:39
QSize:39
QSize:40
QSize:41
QSize:41
QSize:42
QSize:43
QSize:43
QSize:44
QSize:45
QSize:45
QSize:46
QSize:47
QSize:47
QSize:48
QSize:49
QSize:49
QSize:50
QSize:51
QSize:51
QSize:52
QSize:53
QSize:53
QSize:54
QSize:55
QSize:55
QSize:56
QSize:57
QSize:57
QSize:58
QSize:59
QSize:59
QSize:60
QSize:61
QSize:61
QSize:62
QSize:63
QSize:63
QSize:64
QSize:65
QSize:65
QSize:66
QSize:67
QS

KeyboardInterrupt: 

In [2]:
# coding:utf-8
import cv2
import numpy as np
import os,sys,time
from hxnumocr.numrec import OneHotFullLinkNetwork as NumberNet
from colorama import Fore, Back, Style
from collections import deque
import hashlib
from kidgarden import *

time_st = time.time()
narrow_rate_single = 0.15
origin_img, number_core_rects, number_imgs = get_number_grids_from_image("e4.png","imgs",narrow_rate_single=narrow_rate_single)
number_core_rects = np.array(number_core_rects).reshape((16,10,4))
# os.remove(full_file_name)
matrix = build_matrix_from_grid_imgs(number_imgs)
time_end = time.time()
time_cost = time_end - time_st
print(f"Img Recognize : {np.round(time_cost, 2)} s.")

time_st = time.time()
best_solutions = [find_best_solution(matrix, lambda:np.random.randint(6,11),None,True) for _ in range(2)]
time_end = time.time()
time_cost = time_end - time_st  

best_solutions.sort(key=lambda x: (x.score, 0 - len(x.rects)))
best_solution = best_solutions[-1]

Img Recognize : 0.17 s.
Only Reach:106 points with 46 Steps
Only Reach:102 points with 45 Steps


## OpenCV窗口显示红框步数

In [3]:
import cv2

img_cp = origin_img.copy()
narrow_rate_recover = int(((len(number_imgs[0]) / (1 - narrow_rate_single * 2)) * narrow_rate_single) * 1.08)
img_show = cv2.resize(img_cp, (len(img_cp[0]) // 3, len(img_cp) // 3))
cv2.imshow("Steps", img_show)
cv2.waitKey(0)
for step in best_solution.rects + [best_solution.rects[-1]]:
    left_top_rect = number_core_rects[step[0]][step[1]]
    right_bottom_rect = number_core_rects[step[0] + step[2] - 1][step[1] + step[3] - 1]
    lt_y,lt_x,rb_y,rb_x = *left_top_rect[:2], right_bottom_rect[0] + right_bottom_rect[2], right_bottom_rect[1] + right_bottom_rect[3]
    red_rect_temp = img_cp.copy()
    cv2.rectangle(red_rect_temp,(lt_y - narrow_rate_recover, lt_x - narrow_rate_recover), (rb_y + narrow_rate_recover, rb_x + narrow_rate_recover), (0,0,255), thickness=len(number_imgs[0]) // 4)
    img_show = cv2.resize(red_rect_temp, (len(img_cp[0]) // 3, len(img_cp) // 3))
    cv2.imshow("Steps", img_show)
    cv2.waitKey(400)
    cv2.rectangle(img_cp,(lt_y - narrow_rate_recover, lt_x - narrow_rate_recover), (rb_y + narrow_rate_recover, rb_x + narrow_rate_recover), (88,122,61), thickness=-1)
cv2.imshow("Steps", img_show)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
img_cp = origin_img.copy()
img_cp[549][907:977]


array([[ 88, 122,  61],
       [ 88, 122,  61],
       [ 88, 122,  61],
       [ 88, 123,  61],
       [ 91, 127,  62],
       [ 91, 127,  62],
       [ 91, 127,  62],
       [ 89, 124,  61],
       [ 88, 122,  61],
       [ 88, 122,  61],
       [ 88, 122,  61],
       [ 88, 122,  61],
       [ 88, 122,  61],
       [ 88, 122,  61],
       [ 88, 122,  61],
       [ 87, 121,  60],
       [ 85, 118,  58],
       [ 83, 115,  57],
       [ 78, 109,  54],
       [ 77, 106,  53],
       [ 79, 105,  56],
       [160, 171, 154],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252, 254],
       [246, 252

In [5]:
import win32clipboard
import win32con
import cv2
import win32gui
import win32ui
import numpy as np
import time

# Win32API不清晰，放弃
def bmp_handle_2_image(bmp_handle:int):
    bmp_info = win32gui.GetObject(bmp_handle)
    bitmap_obj = win32ui.CreateBitmapFromHandle(bmp_handle)

    bits = bitmap_obj.GetBitmapBits(True)
    img = np.frombuffer(bits).astype(np.uint8)
    img.shape = (bmp_info["bmHeight"], bmp_info["bmWidth"], 4)
    img = img[:,:,:3]

    return img


try:
    # win32clipboard.EmptyClipboard()
    # win32clipboard.OpenClipboard()
    while True:
        win32clipboard.OpenClipboard()
        if win32clipboard.IsClipboardFormatAvailable(win32con.CF_BITMAP):
            win32clipboard.OpenClipboard()
            data = win32clipboard.GetClipboardData(win32con.CF_BITMAP)
            print(data)
            img = bmp_handle_2_image(bmp_handle=data)
            cv2.imwrite("temp/fromclipboard.jpg", img)
        else:
            print("No Img")
        time.sleep(0.5)
finally:
    win32clipboard.CloseClipboard()
    print("Clipboard closed")


No Img
No Img
No Img
No Img
No Img
No Img
No Img
No Img
No Img
No Img
No Img
No Img
No Img
-200992474
Clipboard closed


ValueError: buffer size must be a multiple of element size

In [7]:
from PIL import Image, ImageGrab
import cv2
import numpy as np
import win32clipboard

def clear_clipboard():
    try:
        win32clipboard.OpenClipboard()
        win32clipboard.EmptyClipboard()
        win32clipboard.CloseClipboard()
    except Exception:
        pass
    finally:
        try:
            win32clipboard.CloseClipboard
        except Exception:
            pass

clear_clipboard()
pil_img = None
while True:
    pil_img = ImageGrab.grabclipboard()
    if isinstance(pil_img, Image.Image):
        break
    time.sleep(0.5)

cv_img = np.array(pil_img)
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_RGB2BGR)
cv2.imwrite("temp/clipboardimage.png", cv_img)
print("剪贴板图片已保存")

剪贴板图片已保存


In [1]:
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QPainter
from PyQt5.QtWidgets import QApplication, QLabel, QWidget
import ctypes, sys

class StepWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        self.setGeometry(100,100,400,300)

        self.setWindowOpacity(0.5)

        self.setAttribute(Qt.WA_TranslucentBackground, True)

        # hwnd = self.winId().__init__()
        # style = ctypes.windll.user32.GetWindowLongPtrW(hwnd, -20)
        # ctypes.windll.user32.SetWindowLongPtrW(hwnd, -20, style | 0x80000 | 0x20)

        # self.setWindowFlags(Qt.WindowStaysOnTopHint | Qt.WindowTransparentForInput)
    
    def mousePressEvent(self, event):
        print(event.pos().y())
        if event.pos().y() > 100:
            self.setWindowFlags(Qt.WindowStaysOnTopHint | Qt.WindowTransparentForInput)
        else:
            self.setWindowFlags(Qt.WindowStaysOnTopHint)

        
app = QApplication([])
window = StepWindow()

window.show()

app.exec_()

78


In [1]:
from ctypes import CDLL

adder = CDLL("clib/find_a_solution.dll")

print(adder.add(1,56))

57
